## Goals 

1. 딥러닝 모델 학습에 필요한 이미지 데이터셋을 불러오고, 전처리하는 방법 실습
2. CNN 모델을 구현하고, 학습하는 방법 실습

In [ ]:
import tensorflow as tf
import numpy as np

## 1. 데이터셋 준비하기

### 데이터셋 불러오기

https://www.tensorflow.org/api_docs/python/tf/keras/datasets

- FashionMNIST 데이터셋을 Tensorflow에서 제공하는 API를 통해 로드

    - `x_train`: 학습에 사용되는 이미지 데이터
    - `y_train`: 학습에 사용되는 레이블

    - `x_test`: 성능 평가 (테스트)에 사용되는 이미지 데이터
    - `y_test`: 성능 평가 (테스트)에 사용되는 레이블

In [22]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

### 데이터셋 정보 확인

- 데이터셋을 구성하는 Tensor의 shape 확인
    - Tensorflow의 `Tensor` 객체는 이미지 데이터를 나타내는 3차원 Tensor를 사용 
    - Tensor의 shape은 `(이미지 개수, 이미지 높이, 이미지 너비, 채널 수)`로 구성
    - 컬러 이미지의 경우 채널 수가 3 (RGB) 이며, 흑백 이미지의 경우 채널 수가 1 (따라서 shape에서 생략됨)

In [ ]:
print("아래 셀에서 보이는 것처럼 FashionMNIST 데이터셋은 흑백 이미지로 구성되어 있으므로, 채널 수가 1, 높이와 너비는 각각 28")
print('학습 데이터 shape:', x_train.shape)
print('테스트 데이터 shape:', x_test.shape)

print('학습 라벨 shape:', y_train.shape)
print('테스트 라벨 shape:', y_test.shape)

- 총 10개의 클래스가 존재하며, 각 클래스는 다음과 같은 정수로 레이블링
    - 0: T-shirt/top
    - 1: Trouser
    - 2: Pullover
    - 3: Dress
    - 4: Coat
    - 5: Sandal
    - 6: Shirt
    - 7: Sneaker
    - 8: Bag
    - 9: Ankle boot


In [ ]:
class_names = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

print('클래스 별 학습 데이터 수')
for i in range(10):
    print(f'{i}번 클래스: {class_names[i]}', y_train[y_train == i].shape[0])

- 학습 시간 단축을 위해서 60,000개의 학습 데이터 중 10,000개를 랜덤 추출하여 사용

- numpy의 `np.random.choice` API를 사용하여 랜덤 추출된 인덱스를 저장하는 `indices` 변수를 생성하고, `x_train`과 `y_train`에서 랜덤으로 추출된 10,000개의 데이터를 저장
    -  https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html


In [ ]:
x_train.shape

In [ ]:
# random index sampling (0 ~ 59999 의 정수 중 랜덤으로 10000개를 샘플링)
random_idx = np.random.choice(60000, 10000, replace=False)
print(random_idx)

In [ ]:
# 샘플링된 인덱스로 데이터셋 서브샘플링 (10000개), numpy 어레이 인덱싱 활용
x_train = x_train[random_idx]
y_train = y_train[random_idx]

In [ ]:
print("아래 셀에서 보이는 것처럼 FashionMNIST 데이터셋은 흑백 이미지로 구성되어 있으므로, 채널 수가 1, 높이와 너비는 각각 28")
print('학습 데이터 shape:', x_train.shape)
print('테스트 데이터 shape:', x_test.shape)

print('학습 라벨 shape:', y_train.shape)
print('테스트 라벨 shape:', y_test.shape)

In [ ]:
class_names = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

print('클래스 별 학습 데이터 수')
for i in range(10):
    print(f'{i}번 클래스: {class_names[i]}', y_train[y_train == i].shape[0])

### 데이터 이미지 확인

- `matplotlib` 패키지를 활용한 시각화를 통해 데이터 이미지 확인

In [ ]:
class_names = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

import matplotlib.pyplot as plt

plt.figure(figsize=(3,3))

i = 140
# 학습 데이터셋의 i 번째 이미지를 시각화
image = x_train[i]
label = y_train[i]


plt.imshow(image, cmap='gray')
plt.xlabel(class_names[y_train[i]])
plt.show()

In [ ]:
class_names = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
plt.figure(figsize=(10,10))

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap='gray')
    plt.xlabel(class_names[y_train[i]])
plt.show()

### 데이터 로더 준비

- 이미지 데이터의 픽셀 값은 0~255 사이의 값을 가짐


In [ ]:
print('최대 픽셀 값', x_train.max())
print('최소 픽셀 값', x_train.min())

- 딥러닝 모델 학습 시, 각 픽셀 값 범위를 0~1 사이로 조정

In [32]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
print('최대 픽셀 값', x_train.max())
print('최소 픽셀 값', x_train.min())

- Stochastic (Mini-batch) gradient descent 알고리즘을 통한 모델 학습을 위해 데이터를 적절한 크기로 잘라서 학습에 활용
    - Tensorflow의 `Dataset` 패키지를 활용, 전체 데이터를 배치 단위로 분할

In [34]:
# 학습/테스트 데이터셋 배치 로더 정의

batch_size = 32

tf_train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
tf_test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_batch_loader = tf_train_dataset.shuffle(buffer_size=100000).batch(batch_size)
test_batch_loader = tf_test_dataset.batch(batch_size)

- 배치 단위로 분할된 `train_batch_loader`, `test_batch_loader`는 아래와 같이 반복문을 통해 iterate 가능 

In [ ]:
# 데이터셋의 모든 데이터 (10000개)가 한번씩 뽑힐 때 까지 batch size 만큼 데이터 반환
for x, y in train_batch_loader:
    print(x.shape, y.shape)
    break

# 2-1. CNN 모델 정의하기

### Dense (Fully-Connected) Layer

- 입력 vector $x$ 를 받아 학습 파라미터 $W$와의 $W^Tx = y$ 을 수행하는 레이어 

- 입력 벡터가 $x\in \mathbb{R}^{\text{in}}$, 출력 벡터가 $y\in \mathbb{R}^{\text{out}}$의 차원을 갖는 경우 $W$의 차원은 $\mathbb{R}^{\text{in}\times \text{out}}$

- Dense layer 정의

In [36]:
output_dimension = 128
dense_layer = tf.keras.layers.Dense(units=output_dimension, activation='relu') # y= = Relu(Dense(x))

- 랜덤 입력 데이터 생성

In [ ]:
batch_size = 100
input_dimension = 32
input = tf.random.normal(shape=[batch_size, input_dimension])

print('random input shape: ', input.shape)

- Dense layer 연산 (forward)

In [ ]:
output = dense_layer(input)

In [ ]:
print('output shape: ', output.shape)

- Dense layer의 학습 파라미터 shape

In [ ]:
dense_layer.get_weights()[0].shape

### Convolution Layer

- (height, width, channel)의 shape을 갖는 이미지 데이터를 입력으로 받아, 컨볼루션 필터를 이용해 이미지 데이터에 대한 합성곱 연산을 수행하는 레이어
- 2차원 이미지 데이터를 dense layer가 처리 가능한 1차원 데이터로 변환하기 위해서 Flatten layer 사용

In [41]:
# filters: 출력 채널 수
# kernel_size: 컨볼루션 필터 크기
# padding: 입력 이미지에 부여되는 패딩 규칙 ('same' 일 경우 입력과 출력의 너비와 높이가 변하지 않음)
conv_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')

In [ ]:
batch_size = 100
input_height = 28
input_width = 28
input_channel = 1
input = tf.random.normal(shape=[100, 28, 28, 1])

print('random input shape: ', input.shape)

In [ ]:
output = conv_layer(input)
output.shape

### Pooling Layer

- Convolution layer를 통해 얻어진 출력 (activation map)으로부터 주요한 정보만을 남기고, 불필요한 정보를 제거하는 레이어 (downsampling 수행)
- `pool_size`가 `k` 일 때 `(height, width, channel)`의 shape을 갖는 입력에 대해 `(height/k, width/k, channel)`의 shape을 갖는 출력 반환

In [ ]:
maxpool_layer = tf.keras.layers.MaxPool2D(pool_size=2)

In [ ]:
batch_size = 100
input_height = 28
input_width = 28
input_channel = 1
input = tf.random.normal(shape=[100, 28, 28, 1])

print('random input shape: ', input.shape)

In [ ]:
output = maxpool_layer(input)
output.shape

TensorShape([100, 9, 9, 1])

### CNN 모델 정의

- Convolution layer와 pooling layer, dense layer를 조합하여 CNN 모델 정의
- 정의된 CNN 모델의 **출력 차원은 학습 데이터의 클래스 수**와 일치 해야함
    - Fashion MNIST 데이터셋의 경우 총 10개의 클래스 존재

**Note**
- 마지막에 Dense layer - Softmax layer가 와야함
- 이 때 Dense layer 의 출력 차원은 class 개수와 같아야함

In [ ]:
from tensorflow.keras import Model

class SingleLayerCNN(Model):
  def __init__(self): # 생성자: 객체가 생성될 때 자동으로 호출되는 메서드 (모델을 구성하는 layer들을 정의)
    super(SingleLayerCNN, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(16, 3, activation='relu')
    self.maxpool1 = tf.keras.layers.MaxPool2D(2)

    self.flatten = tf.keras.layers.Flatten() # 다차원 출력을 1차원 벡터로 변환 (Dense layer에 연결하기 위해 필요한 작업)
    
    self.dense = tf.keras.layers.Dense(10) # 10개의 클래스에 대한 확률값을 출력하기 위해 10개의 뉴런을 가지는 Dense layer를 정의
    self.softmax = tf.keras.layers.Softmax() # 확률값을 계산하기 위해 softmax 함수를 정의    

  def call(self, x): # 모델이 호출될 때 자동으로 호출되는 메서드 (모델에 입력이 들어왔을 때의 레이어 연산 순서 정의)
    x = self.conv1(x)
    x = self.maxpool1(x)
    
    x = self.flatten(x)
    x = self.dense(x)

    x = self.softmax(x)

    return x

### Quiz 1

- **Goal**: 위의 SingleLayerCNN에 convolution layer와 maxpooling layer를 하나씩 추가해서 TwoLayerCNN 모델을 정의하세요
  
- **TODO 1**: 아래 `TwoLayerCNN`의 `__init__` 메소드에서 `tf.keras.layers.Conv2D`와 `tf.keras.layers.MaxPool2D`를 사용하여 convolution layer와 maxpooling layer를 추가 선언
- **TODO 2**: 아래 `TwoLayerCNN`의 `call` 메소드에서 `self.conv1`과 `self.pool1`을 통과한 출력을 새로 선언한 convolution layer와 maxpooling layer에 순차적으로 입력

    


In [ ]:
from tensorflow.keras import Model

class TwoLayerCNN(Model):
  def __init__(self): # 생성자: 객체가 생성될 때 자동으로 호출되는 메서드 (모델을 구성하는 layer들을 정의)
    super(TwoLayerCNN, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(16, 3, activation='relu')
    self.maxpool1 = tf.keras.layers.MaxPool2D(2)

    # TODO 1




    self.flatten = tf.keras.layers.Flatten() # 다차원 출력을 1차원 벡터로 변환 (Dense layer에 연결하기 위해 필요한 작업)
    
    self.dense = tf.keras.layers.Dense(10) # 10개의 클래스에 대한 확률값을 출력하기 위해 10개의 뉴런을 가지는 Dense layer를 정의
    self.softmax = tf.keras.layers.Softmax() # 확률값을 계산하기 위해 softmax 함수를 정의    

  def call(self, x):
    x = self.conv1(x)
    x = self.maxpool1(x)
    
    # TODO 2



    
    x = self.flatten(x)
    x = self.dense(x)

    x = self.softmax(x)

    return x

# 2-2. 모델 학습

### 모델 인스턴스 생성

- 입력 shape을 지정하여 model build 수행

In [46]:
model = TwoLayerCNN()
input_shape = (1,28,28,1)
model.build(input_shape)

In [ ]:
model.summary()

### 모델 학습을 위한 loss 함수와 optimizer 정의

- Tensorflow 에서 지원하는 Loss functions: https://www.tensorflow.org/api_docs/python/tf/keras/losses

    - `CategoricalCrossentropy` : label이 onehot encoding (ex. [0, 0, 0, 1]) 형태로 제공되는 경우 사용
    - `SparseCategoricalCrossentropy` : label이 integer 형태로 제공되는 경우 사용

- Tensorflow 에서 지원하는 Optimizers: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers

In [54]:
# loss 함수 (분류 모델 학습을 위한 Cross Entropy Loss)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

# loss 함수를 minimize 하기 위한 optimizer
optimizer = tf.keras.optimizers.Adam() #default arguments: lr: 0.001, beta_1: 0.9, beta_2: 0.999, epsilon: 1e-07

### 학습 및 추론 함수 작성

학습에 필요한 gradient 값을 획득하기 위해서 Tensorflow에서 제공하는 `GradientTape` 객체 사용

- `GradientTape()` 활용 예시
    - $y=x^2$ 함수에 대한 미분을 계산하는 예시

In [ ]:
x = tf.constant(3.0) # x 변수에 상수 값 3.0 할당

with tf.GradientTape() as tape: # gradient 계산을 시작하기 위한 with statement
  tape.watch(x) # x 변수에 대한 gradient 계산을 위해 x를 watch
  y = x * x # y = x^2

dy_dx = tape.gradient(y, x) # take the gradient of y with respect to x (dy/dx = 2x)
print(f'x={x}\t', 'dy/dx=2x\t ', dy_dx)

- 학습 함수 정의

In [58]:
def train(model, train_batch_loader, loss_object, optimizer):
  for images, labels in train_batch_loader: # train_batch_loader로부터 batch (images, labels)를 가져옴
    with tf.GradientTape() as tape: # 그래디언트 계산 시작
      outputs = model(images) # batch 단위 이미지를 모델에 입력하여 출력 도출 (Forward)
      loss = loss_object(labels, outputs) # 정답 라벨 labels와 모델의 출력 outputs (예측치) 간의 loss 계산
    gradients = tape.gradient(loss, model.trainable_variables) # gradient of loss with respect to weight parameters (Back-propagation)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables)) # Adam optimizer를 이용하여 weight parameter 업데이트 


# => 전체 학습 데이터셋 (10,000개)에 대해 학습을 수행함 (1 Epoch)

- 성능 평가 (test) 함수 정의

In [ ]:
!pip install scikit_learn

In [59]:
from sklearn.metrics import accuracy_score # scikit learn 패키지의 accuracy_score 함수를 이용하여 정확도 계산

def test(model, test_batch_loader):
    
    preds_list = [] # 모델의 예측치를 저장하기 위한 list
    labels_list = [] # 정답 라벨을 저장하기 위한 list

    for images, labels in test_batch_loader:
        outputs = model(images) # batch 단위 이미지를 모델에 입력하여 출력 도출 (Forward)

        outputs = outputs.numpy() # 편의를 위해 numpy array로 변환
        labels = labels.numpy()

        preds = outputs.argmax(axis=1) # 모델의 예측치는 확률값이 가장 큰 클래스 (ex) [0.1, 0.2, 0.7] => 2)로 결정
        
        preds_list += list(preds) # 예측치를 list에 추가
        labels_list += list(labels) # 정답 라벨을 list에 추가

    accuracy = accuracy_score(labels_list, preds_list)
    return accuracy

    

### Quiz 2

- **Goal**: 위의 학습 함수와 성능평가 함수를 이용하여 모델을 학습하고, 성능을 평가하세요.
  
- **TODO 1**: `train` 함수를 활용하여 모델을 3 epoch 만큼 학습시키키
- **TODO 2**: `test` 함수를 활용하여 학습된 모델 성능을 평가하기




In [65]:
# TODO 1


In [66]:
# TODO 2
